Importing modules and libraries

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [10]:
df = pd.read_csv(
    "../data/cleaned/ucsb_cleaned_listings_combo.csv",
    encoding = "latin1"
)

df.head()

,address,bathrooms,bedrooms,price_from,distance_to_ucsb_miles
0,"100 Baldwin Dr, Unit 107, Goleta, CA",1.0,0,3255,1.8
1,"100 Baldwin Dr, Unit 115, Goleta, CA",1.0,0,3240,1.8
2,"100 Baldwin Dr, Unit 205, Goleta, CA",1.0,1,3310,1.8
3,"100 Baldwin Dr, Unit 207, Goleta, CA",1.0,1,3310,1.8
4,"100 Baldwin Dr, Unit 203, Goleta, CA",1.0,1,3770,1.8
